In [ ]:

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])
x_train, x_test = x_train / 255., x_test / 255.

# 학습에 필요한 설정값들을 정의합니다.
learning_rate = 0.02
training_epochs = 50    # 반복횟수
batch_size = 256        # 배치개수
display_step = 1        # 손실함수 출력 주기
examples_to_show = 10   # 보여줄 MNIST Reconstruction 이미지 개수
input_size = 784        # 28*28
hidden1_size = 256
hidden2_size = 128

# tf.data API를 이용해서 데이터를 섞고 batch 형태로 가져옵니다.
train_data = tf.data.Dataset.from_tensor_slices(x_train)
train_data = train_data.shuffle(60000).batch(batch_size)

# Autoencoder 모델을 정의합니다.
class AutoEncoder(object):
    # Autoencoder 모델을 위한 tf.Variable들을 정의합니다.
    def __init__(self):
        # 인코딩(Encoding) - 784 -> 256 -> 128
        self.W1 = tf.Variable(tf.random.normal(shape=[input_size, hidden1_size]))
        self.b1 = tf.Variable(tf.random.normal(shape=[hidden1_size]))
        self.W2 = tf.Variable(tf.random.normal(shape=[hidden1_size, hidden2_size]))
        self.b2 = tf.Variable(tf.random.normal(shape=[hidden2_size]))
        # 디코딩(Decoding) 128 -> 256 -> 784
        self.W3 = tf.Variable(tf.random.normal(shape=[hidden2_size, hidden1_size]))
        self.b3 = tf.Variable(tf.random.normal(shape=[hidden1_size]))
        self.W4 = tf.Variable(tf.random.normal(shape=[hidden1_size, input_size]))
        self.b4 = tf.Variable(tf.random.normal(shape=[input_size]))

    def __call__(self, x):
        H1_output = tf.nn.sigmoid(tf.matmul(x, self.W1) + self.b1)
        H2_output = tf.nn.sigmoid(tf.matmul(H1_output, self.W2) + self.b2)
        H3_output = tf.nn.sigmoid(tf.matmul(H2_output, self.W3) + self.b3)
        reconstructed_x = tf.nn.sigmoid(tf.matmul(H3_output, self.W4) + self.b4)

        return reconstructed_x

# MSE 손실 함수를 정의합니다.
@tf.function
def mse_loss(y_pred, y_true):
    return tf.reduce_mean(tf.pow(y_true - y_pred, 2)) # MSE(Mean of Squared Error) 손실함수

# 최적화를 위한 RMSProp 옵티마이저를 정의합니다.
optimizer = tf.optimizers.RMSprop(learning_rate)

# 최적화를 위한 function을 정의합니다.
@tf.function
def train_step(model, x):
    y_true = x
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = mse_loss(y_pred, y_true)
    gradients = tape.gradient(loss, vars(model).values())
    optimizer.apply_gradients(zip(gradients, vars(model).values()))

AutoEncoder_model = AutoEncoder()

for epoch in range(training_epochs):
    for batch_x in train_data:
        _, current_loss = train_step(AutoEncoder_model, batch_x), mse_loss(AutoEncoder_model(batch_x), batch_x)
    if epoch % display_step == 0:
        print("반복(Epoch): %d, 손실 함수(Loss): %f" % ((epoch+1), current_loss))

reconstructed_result = AutoEncoder_model(x_test[:examples_to_show])
f, a = plt.subplots(2, 10, figsize=(10, 2))
for i in range(examples_to_show):
    a[0][i].imshow(np.reshape(x_test[i], (28, 28)))
    a[1][i].imshow(np.reshape(reconstructed_result[i], (28, 28)))
f.savefig('reconstructed_mnist_image.png')  # reconstruction 결과를 png로 저장합니다.
f.show()
plt.draw()
plt.waitforbuttonpress()

반복(Epoch): 1, 손실 함수(Loss): 0.066458
반복(Epoch): 2, 손실 함수(Loss): 0.053417
반복(Epoch): 3, 손실 함수(Loss): 0.048841
반복(Epoch): 4, 손실 함수(Loss): 0.051335
반복(Epoch): 5, 손실 함수(Loss): 0.046059
반복(Epoch): 6, 손실 함수(Loss): 0.039696
반복(Epoch): 7, 손실 함수(Loss): 0.035320
반복(Epoch): 8, 손실 함수(Loss): 0.035885
반복(Epoch): 9, 손실 함수(Loss): 0.041328
반복(Epoch): 10, 손실 함수(Loss): 0.035479
반복(Epoch): 11, 손실 함수(Loss): 0.033553
반복(Epoch): 12, 손실 함수(Loss): 0.034147
반복(Epoch): 13, 손실 함수(Loss): 0.036413
반복(Epoch): 14, 손실 함수(Loss): 0.030665
반복(Epoch): 15, 손실 함수(Loss): 0.033856
반복(Epoch): 16, 손실 함수(Loss): 0.031700
반복(Epoch): 17, 손실 함수(Loss): 0.028617
반복(Epoch): 18, 손실 함수(Loss): 0.030114
반복(Epoch): 19, 손실 함수(Loss): 0.032127
반복(Epoch): 20, 손실 함수(Loss): 0.027194
반복(Epoch): 21, 손실 함수(Loss): 0.027256
반복(Epoch): 22, 손실 함수(Loss): 0.031238
반복(Epoch): 23, 손실 함수(Loss): 0.028044
반복(Epoch): 24, 손실 함수(Loss): 0.025758
반복(Epoch): 25, 손실 함수(Loss): 0.027937
반복(Epoch): 26, 손실 함수(Loss): 0.025885
반복(Epoch): 27, 손실 함수(Loss): 0.026314
반복(Epoch):

/Users/kimginam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:79: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
/Users/kimginam/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
